In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *

In [4]:
torch.cuda.set_device(0)

In [5]:
PATH = "/home/paperspace/data/dog-breed-identification/"
sz=224
arch=resnext101_64
batch_size=58

In [6]:
!ls {PATH}

labels.csv  sample_submission.csv  test  train


In [7]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)  # random 20% of the rows for training set
label_df = pd.read_csv(label_csv)

In [8]:
label_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [12]:
def get_data(sz, bs):
    tfms = tfms_from_model(
        arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(
        PATH,
        'train',
        label_csv,
        test_name='test',
        val_idxs=val_idxs,
        suffix='.jpg',
        tfms=tfms,
        bs=batch_size
    )
    return data if sz > 300 else data.resize(340, 'tmp')

In [15]:
data = get_data(sz, batch_size)

In [20]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 179/179 [02:07<00:00,  1.40it/s]


In [23]:
learn.fit(lrs=1e-2, n_cycle=5)

[0.      0.96672 0.37756 0.91906]                            
[1.      0.44078 0.37733 0.9114 ]                            
[2.      0.3072  0.29621 0.91236]                            
[3.      0.23339 0.2851  0.91763]                            
[4.      0.20644 0.3097  0.92146]                            



## Disabling Precomputing 

This still trains the last two layers, but activations are now recalculated for repeated images. This means that data augmentation now works!

In [24]:
learn.precompute=False

In [27]:
learn.fit(lrs=1e-2, n_cycle=5, cycle_len=1)

[0.      0.24176 0.23684 0.92768]                            
[1.      0.24799 0.23467 0.92816]                            
[2.      0.23083 0.23474 0.92624]                            
[3.      0.21546 0.23291 0.93056]                            
[4.      0.20346 0.22676 0.92577]                            



In [28]:
learn.save('224_pre')
learn.load('224_pre')

## Increasing the Size

Here we are getting images of a larger size, which will help generalize the model on top of providing more data

In [31]:
learn.set_data(get_data(299, batch_size))
learn.freeze()  # freeze the last layer

## Fiddling with Epochs

In order to more effectively utilize stochastic descent with restarts, we set `cycle_mult=2`. The first cycle is 1 epoch long, the second is 2 epochs long, and then third is 4 epochs long. Doing so prevents the network from "getting suck" in local minima

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)


  3%|▎         | 4/141 [00:06<03:41,  1.62s/it, loss=0.32]

 45%|████▍     | 63/141 [01:24<01:44,  1.34s/it, loss=0.208]


 90%|█████████ | 127/141 [02:45<00:18,  1.31s/it, loss=0.206]


100%|██████████| 141/141 [03:03<00:00,  1.30s/it, loss=0.212]
                                                             [0.      0.21239 0.2217  0.92864]

 35%|███▍      | 49/141 [01:05<02:02,  1.33s/it, loss=0.207]


 79%|███████▉  | 112/141 [02:27<00:38,  1.32s/it, loss=0.209]


100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.204]
                                                             [1.      0.20351 0.22484 0.92768]

 24%|██▍       | 34/141 [00:45<02:24,  1.35s/it, loss=0.181]


...

 59%|█████▉    | 83/141 [01:50<01:17,  1.33s/it, loss=0.183]


100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.182]
                                                             [3.      0.18204 0.22202 0.93103]

  3%|▎         | 4/141 [00:07<04:19,  1.90s/it, loss=0.18] 


 48%|████▊     | 68/141 [01:30<01:37,  1.34s/it, loss=0.173]


 94%|█████████▎| 132/141 [02:52<00:11,  1.31s/it, loss=0.169]


100%|██████████| 141/141 [03:04<00:00,  1.31s/it, loss=0.162]
                                                             [4.      0.16214 0.22011 0.93151]

  9%|▊         | 12/141 [00:16<03:00,  1.40s/it, loss=0.159]

In [ ]:
learn.save('299_pre')

In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y),  metrics.log_loss(y, probs)

## Creating a Kaggle-Compatible Submission

In [ ]:
log_preds, y = learn.TTA(is_test=True)